In [ ]:
##### Cleaning the softskill annotated data

In [2]:
import pandas as pd
import spacy
import csv
#
from tqdm import tqdm
tqdm.pandas()

C:\Users\Deligram\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
def character_counter(text):
    from collections import Counter
    my_str = text
    counter = Counter(my_str)
    return counter['*']

In [20]:
nlp = spacy.load("en_core_web_sm")

In [5]:
df1 = pd.read_csv('job_test.csv')
df1 = pd.DataFrame(df1, columns=['is_soft_skill', 'soft_skill', 'context', 'test'])
df1['test'] = df1['test'].fillna('')
df1['context'] = df1['context'] + ' ' + df1['test']
df1 = df1[['is_soft_skill', 'soft_skill', 'context']]
df1 = df1.drop(index=0)

In [11]:
df2 = pd.read_csv('job_train.csv')
df2 = pd.DataFrame(df2, columns=['is_soft_skill', 'soft_skill', 'context'])
df2 = df2.drop(index=0)

,is_soft_skill,soft_skill,context
1,1,team member,look for to work a a xxx xxx with hilton hotel s
2,1,verbal communication skills,"to detail and accuracy , fantastic write and x..."
3,0,dynamic,for an experience and dedicate ict manager to ...
4,0,dynamic,"this be a unique , xxx and challenge role that..."
5,0,independent,the home be an xxx care home with a great repu...
...,...,...,...
3700,1,ability to compile a report,strong understand of legislation best practice...
3701,0,dynamic,"to provide fix report , interactive report bui..."
3702,1,political awareness,conduct and respect posse cultural and xxx xxx...
3703,1,leadership skills,high level of experience and xxx xxx be essent...


In [12]:
df = pd.concat([df1,df2]).reset_index()

,index,is_soft_skill,soft_skill,context
0,1,1,team customer interaction,effective member of the operation leadership x...
1,2,1,systematic,linux shell be able to demonstrate a xxx and a...
2,3,1,autonomy,territory manager role offer significant respo...
3,4,1,hard working sales person,"you must be an enthuse , passionate , xxx xxx ..."
4,5,1,team s commitment,effective relationship with and obtain the int...
...,...,...,...,...
7404,3700,1,ability to compile a report,strong understand of legislation best practice...
7405,3701,0,dynamic,"to provide fix report , interactive report bui..."
7406,3702,1,political awareness,conduct and respect posse cultural and xxx xxx...
7407,3703,1,leadership skills,high level of experience and xxx xxx be essent...


In [14]:
df['test'] = df['context'].str.replace(r'[^A-Za-z0-9 ]+', ' ')
df['test'] = df['test'].str.replace(r'xxx ', 'XXX', regex=True)
df['test'] = df['test'].str.replace(r'(X+)', '* ', regex=True)
df['test'] = df['test'].str.strip()
df['test'] = df['test'].str.lower()
df['test'] = df['test'].str.replace(r'[ ]{2,}', ' ', regex=True)

df['test2'] = df['test'].apply(character_counter)

#Removing rows which have multiple soft skills 
df = df[df.test2==1]

In [15]:
final = df['test'].str.split('*',expand=True)
df = df.join(final)
df_final = df[[0,'soft_skill',1, 'is_soft_skill']]
df_final = df_final.rename(columns={0:'left_context', 1:'right_context'})

df_final = df_final.rename(columns={'soft_skill' : 'candidate_skill', 'is_soft_skill':'label'})

,index,is_soft_skill,soft_skill,context,test,test2,0,1
0,1,1,team customer interaction,effective member of the operation leadership x...,effective member of the operation leadership *...,1,effective member of the operation leadership,to develop positive relationship a
1,2,1,systematic,linux shell be able to demonstrate a xxx and a...,linux shell be able to demonstrate a * and ana...,1,linux shell be able to demonstrate a,and analytical approach to
2,3,1,autonomy,territory manager role offer significant respo...,territory manager role offer significant respo...,1,territory manager role offer significant respo...,
3,4,1,hard working sales person,"you must be an enthuse , passionate , xxx xxx ...",you must be an enthuse passionate * with a fle...,1,you must be an enthuse passionate,with a flexible attitude
4,5,1,team s commitment,effective relationship with and obtain the int...,effective relationship with and obtain the int...,1,effective relationship with and obtain the int...,to i document and
...,...,...,...,...,...,...,...,...
7404,3700,1,ability to compile a report,strong understand of legislation best practice...,strong understand of legislation best practice...,1,strong understand of legislation best practice,provide a clinical opinion
7405,3701,0,dynamic,"to provide fix report , interactive report bui...",to provide fix report interactive report build...,1,to provide fix report interactive report build...,legal document creation
7406,3702,1,political awareness,conduct and respect posse cultural and xxx xxx...,conduct and respect posse cultural and * and s...,1,conduct and respect posse cultural and,and sensitivity exercise flexiblity
7407,3703,1,leadership skills,high level of experience and xxx xxx be essent...,high level of experience and * be essential a ...,1,high level of experience and,be essential a well


In [21]:
#Removing candidate skills with more than 4 tokens in order to standardize the data with the manually labeled data

df_final['length'] = df_final['candidate_skill'].apply(nlp)
df_final['length'] = df_final['length'].apply(len)
df_final = df_final[df_final.length<=4]
df_final

,left_context,candidate_skill,right_context,label,length
0,effective member of the operation leadership,team customer interaction,to develop positive relationship a,1,3
1,linux shell be able to demonstrate a,systematic,and analytical approach to,1,1
2,territory manager role offer significant respo...,autonomy,,1,1
3,you must be an enthuse passionate,hard working sales person,with a flexible attitude,1,4
4,effective relationship with and obtain the int...,team s commitment,to i document and,1,3
...,...,...,...,...,...
7403,a fun and,dynamic,opportunity that offer fantastic career progr...,0,1
7405,to provide fix report interactive report build...,dynamic,legal document creation,0,1
7406,conduct and respect posse cultural and,political awareness,and sensitivity exercise flexiblity,1,2
7407,high level of experience and,leadership skills,be essential a well,1,2


In [22]:
# df_final.to_csv("annotated_soft_skills_context.csv")